In [1]:
import geopandas as gpd
import pandas as pd
import json

In [2]:
countries = gpd.read_file('./countries.geojson')

In [3]:
countries

,ADMIN,ISO_A3,ISO_A2,geometry
0,Aruba,ABW,AW,"MULTIPOLYGON (((-69.99694 12.57758, -69.93639 ..."
1,Afghanistan,AFG,AF,"MULTIPOLYGON (((71.04980 38.40866, 71.05714 38..."
2,Angola,AGO,AO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
3,Anguilla,AIA,AI,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
4,Albania,ALB,AL,"MULTIPOLYGON (((19.74777 42.57890, 19.74601 42..."
...,...,...,...,...
250,Samoa,WSM,WS,"MULTIPOLYGON (((-171.57002 -13.93816, -171.564..."
251,Yemen,YEM,YE,"MULTIPOLYGON (((53.30824 12.11839, 53.31027 12..."
252,South Africa,ZAF,ZA,"MULTIPOLYGON (((37.86378 -46.94085, 37.83644 -..."
253,Zambia,ZMB,ZM,"MULTIPOLYGON (((31.11984 -8.61663, 31.14102 -8..."


# Integrate Arabica Ratings

In [4]:
arabica = pd.read_csv('processed_coffee_ratings_arabica.csv')
arabica['Harvest.Year'] = arabica['Harvest.Year'].astype(str)

arabica

,Country.of.Origin,Harvest.Year,Aroma,Flavor,Aftertaste,Acidity,Body,Balance,Uniformity,Clean.Cup,Sweetness,Cupper.Points,Total.Cup.Points
0,Brazil,2010,8.295000,7.920000,7.960000,7.750000,8.125000,8.000000,10.000000,10.000000,10.000000,8.125000,86.170000
1,Brazil,2011,8.017143,7.929286,7.684286,7.726429,7.690714,7.737857,10.000000,10.000000,10.000000,7.732857,84.517857
2,Brazil,2012,7.625000,7.600000,7.408500,7.500500,7.508000,7.513000,9.966500,9.966500,9.966500,7.497000,82.550500
3,Brazil,2013,7.647500,7.627500,7.460000,7.625000,7.667500,7.832500,9.832500,9.832500,10.000000,7.542500,83.062500
4,Brazil,2014,7.451429,7.500000,7.488571,7.440000,7.512857,7.522857,9.904286,9.428571,9.904286,7.500000,81.655714
...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,United States (Puerto Rico),2013,7.647500,7.540000,7.480000,7.602500,7.665000,7.647500,9.000000,9.665000,9.832500,7.647500,81.727500
125,Vietnam,2012,7.670000,7.665000,7.170000,7.330000,7.875000,7.250000,10.000000,10.000000,10.000000,7.545000,82.500000
126,Vietnam,2014,7.333333,7.443333,7.470000,7.500000,7.556667,7.473333,9.776667,10.000000,10.000000,7.973333,82.526667
127,Vietnam,2017,7.500000,7.335000,7.165000,7.455000,7.500000,7.955000,9.335000,10.000000,10.000000,7.415000,81.670000


In [5]:
# Define the coffee characteristics we're interested in
coffee_characteristics = [
    'Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body', 'Balance', 
    'Uniformity', 'Clean.Cup', 'Sweetness', 'Cupper.Points', 'Total.Cup.Points'
]

def create_yearly_dicts(row):
    return {
        'Year': row['Harvest.Year'],
        'Data': {
            'Aroma': row['Aroma'],
            'Flavor': row['Flavor'],
            'Aftertaste': row['Aftertaste'],
            'Acidity': row['Acidity'],
            'Body': row['Body'],
            'Balance': row['Balance'],
            'Uniformity': row['Uniformity'],
            'Clean.Cup': row['Clean.Cup'],
            'Sweetness': row['Sweetness'],
            'Cupper.Points': row['Cupper.Points'],
            'Total.Cup.Points': row['Total.Cup.Points']
        }
    }


In [6]:
def create_production_dicts(row):
    return {
        'Year': row['Year'],
        'Data': {
            'Arabica Production': row['Arabica Production'],
            'Bean Exports': row['Bean Exports'],
            'Bean Imports': row['Bean Imports'],
            'Beginning Stocks': row['Beginning Stocks'],
            'Domestic Consumption': row['Domestic Consumption'],
            'Ending Stocks': row['Ending Stocks'],
            'Exports': row['Exports'],
            'Imports': row['Imports'],
            'Other Production': row['Other Production'],
            'Production': row['Production'],
            'Roast & Ground Exports': row['Roast & Ground Exports'],
            'Roast & Ground Imports': row['Roast & Ground Imports'],
            'Robusta Production': row['Robusta Production'],
            'Rst,Ground Dom. Consum': row['Rst,Ground Dom. Consum'],
            'Soluble Dom. Cons.': row['Soluble Dom. Cons.'],
            'Soluble Exports': row['Soluble Exports'],
            'Soluble Imports': row['Soluble Imports'],
            'Total Distribution': row['Total Distribution'],
            'Total Supply': row['Total Supply']
        }
    }

In [7]:
arabica['ratings'] = arabica.apply(create_yearly_dicts, axis=1)


In [8]:
# Rename columns in arabica to match those in data
arabica = arabica.rename(columns={
    'Country.of.Origin': 'Country',
    'Harvest.Year': 'Year'
})

In [9]:
arabica

,Country,Year,Aroma,Flavor,Aftertaste,Acidity,Body,Balance,Uniformity,Clean.Cup,Sweetness,Cupper.Points,Total.Cup.Points,ratings
0,Brazil,2010,8.295000,7.920000,7.960000,7.750000,8.125000,8.000000,10.000000,10.000000,10.000000,8.125000,86.170000,"{'Year': '2010', 'Data': {'Aroma': 8.295, 'Fla..."
1,Brazil,2011,8.017143,7.929286,7.684286,7.726429,7.690714,7.737857,10.000000,10.000000,10.000000,7.732857,84.517857,"{'Year': '2011', 'Data': {'Aroma': 8.017142857..."
2,Brazil,2012,7.625000,7.600000,7.408500,7.500500,7.508000,7.513000,9.966500,9.966500,9.966500,7.497000,82.550500,"{'Year': '2012', 'Data': {'Aroma': 7.625, 'Fla..."
3,Brazil,2013,7.647500,7.627500,7.460000,7.625000,7.667500,7.832500,9.832500,9.832500,10.000000,7.542500,83.062500,"{'Year': '2013', 'Data': {'Aroma': 7.6475, 'Fl..."
4,Brazil,2014,7.451429,7.500000,7.488571,7.440000,7.512857,7.522857,9.904286,9.428571,9.904286,7.500000,81.655714,"{'Year': '2014', 'Data': {'Aroma': 7.451428571..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,United States (Puerto Rico),2013,7.647500,7.540000,7.480000,7.602500,7.665000,7.647500,9.000000,9.665000,9.832500,7.647500,81.727500,"{'Year': '2013', 'Data': {'Aroma': 7.6475, 'Fl..."
125,Vietnam,2012,7.670000,7.665000,7.170000,7.330000,7.875000,7.250000,10.000000,10.000000,10.000000,7.545000,82.500000,"{'Year': '2012', 'Data': {'Aroma': 7.67, 'Flav..."
126,Vietnam,2014,7.333333,7.443333,7.470000,7.500000,7.556667,7.473333,9.776667,10.000000,10.000000,7.973333,82.526667,"{'Year': '2014', 'Data': {'Aroma': 7.333333333..."
127,Vietnam,2017,7.500000,7.335000,7.165000,7.455000,7.500000,7.955000,9.335000,10.000000,10.000000,7.415000,81.670000,"{'Year': '2017', 'Data': {'Aroma': 7.5, 'Flavo..."


In [10]:
data = pd.read_csv("./psd_coffee.csv")



In [11]:
# Apply the function to the data DataFrame
data['ratings'] = data.apply(create_production_dicts, axis=1)

In [12]:
data

,Country,Year,Arabica Production,Bean Exports,Bean Imports,Beginning Stocks,Domestic Consumption,Ending Stocks,Exports,Imports,...,Roast & Ground Exports,Roast & Ground Imports,Robusta Production,"Rst,Ground Dom. Consum",Soluble Dom. Cons.,Soluble Exports,Soluble Imports,Total Distribution,Total Supply,ratings
0,Albania,1960,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"{'Year': 1960, 'Data': {'Arabica Production': ..."
1,Albania,1961,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"{'Year': 1961, 'Data': {'Arabica Production': ..."
2,Albania,1962,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"{'Year': 1962, 'Data': {'Arabica Production': ..."
3,Albania,1963,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"{'Year': 1963, 'Data': {'Arabica Production': ..."
4,Albania,1964,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"{'Year': 1964, 'Data': {'Arabica Production': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6011,Zimbabwe,2019,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"{'Year': 2019, 'Data': {'Arabica Production': ..."
6012,Zimbabwe,2020,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"{'Year': 2020, 'Data': {'Arabica Production': ..."
6013,Zimbabwe,2021,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"{'Year': 2021, 'Data': {'Arabica Production': ..."
6014,Zimbabwe,2022,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"{'Year': 2022, 'Data': {'Arabica Production': ..."


In [13]:
arabica['Year'] = arabica['Year'].astype(int)


In [14]:
merged_df = pd.merge(data, arabica, on=['Country', 'Year'], how='outer')


In [15]:
merged_df

,Country,Year,Arabica Production,Bean Exports,Bean Imports,Beginning Stocks,Domestic Consumption,Ending Stocks,Exports,Imports,...,Aftertaste,Acidity,Body,Balance,Uniformity,Clean.Cup,Sweetness,Cupper.Points,Total.Cup.Points,ratings_y
0,Albania,1960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albania,1961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Albania,1962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,1963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Albania,1964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6026,Zimbabwe,2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6027,Zimbabwe,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6028,Zimbabwe,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6029,Zimbabwe,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Combine the ratings data into a single column
def combine_ratings(row):
    combined_data = {}
    if pd.notna(row['ratings_x']):
        combined_data.update(row['ratings_x']['Data'])
    if pd.notna(row['ratings_y']):
        combined_data.update(row['ratings_y']['Data'])
    return {
        'Year': row['Year'],
        'Data': combined_data
    }

In [17]:
merged_df['ratings'] = merged_df.apply(combine_ratings, axis=1)


In [18]:
merged_df

,Country,Year,Arabica Production,Bean Exports,Bean Imports,Beginning Stocks,Domestic Consumption,Ending Stocks,Exports,Imports,...,Acidity,Body,Balance,Uniformity,Clean.Cup,Sweetness,Cupper.Points,Total.Cup.Points,ratings_y,ratings
0,Albania,1960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Year': 1960, 'Data': {'Arabica Production': ..."
1,Albania,1961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Year': 1961, 'Data': {'Arabica Production': ..."
2,Albania,1962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Year': 1962, 'Data': {'Arabica Production': ..."
3,Albania,1963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Year': 1963, 'Data': {'Arabica Production': ..."
4,Albania,1964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Year': 1964, 'Data': {'Arabica Production': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6026,Zimbabwe,2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Year': 2019, 'Data': {'Arabica Production': ..."
6027,Zimbabwe,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Year': 2020, 'Data': {'Arabica Production': ..."
6028,Zimbabwe,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Year': 2021, 'Data': {'Arabica Production': ..."
6029,Zimbabwe,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Year': 2022, 'Data': {'Arabica Production': ..."


In [19]:
yearly_data_by_country = merged_df.groupby('Country')['ratings'].apply(list).reset_index()


In [20]:
yearly_data_by_country

,Country,ratings
0,Albania,"[{'Year': 1960, 'Data': {'Arabica Production':..."
1,Algeria,"[{'Year': 1960, 'Data': {'Arabica Production':..."
2,Angola,"[{'Year': 1960, 'Data': {'Arabica Production':..."
3,Argentina,"[{'Year': 1960, 'Data': {'Arabica Production':..."
4,Armenia,"[{'Year': 1960, 'Data': {'Arabica Production':..."
...,...,...
95,Vietnam,"[{'Year': 1960, 'Data': {'Arabica Production':..."
96,Yemen,"[{'Year': 1960, 'Data': {'Arabica Production':..."
97,Yemen (Sanaa),"[{'Year': 1960, 'Data': {'Arabica Production':..."
98,Zambia,"[{'Year': 1960, 'Data': {'Arabica Production':..."


In [21]:
yearly_data_by_country

,Country,ratings
0,Albania,"[{'Year': 1960, 'Data': {'Arabica Production':..."
1,Algeria,"[{'Year': 1960, 'Data': {'Arabica Production':..."
2,Angola,"[{'Year': 1960, 'Data': {'Arabica Production':..."
3,Argentina,"[{'Year': 1960, 'Data': {'Arabica Production':..."
4,Armenia,"[{'Year': 1960, 'Data': {'Arabica Production':..."
...,...,...
95,Vietnam,"[{'Year': 1960, 'Data': {'Arabica Production':..."
96,Yemen,"[{'Year': 1960, 'Data': {'Arabica Production':..."
97,Yemen (Sanaa),"[{'Year': 1960, 'Data': {'Arabica Production':..."
98,Zambia,"[{'Year': 1960, 'Data': {'Arabica Production':..."


In [22]:
merged_gdf = countries.merge(yearly_data_by_country, left_on='ADMIN', right_on='Country', how='left')
merged_gdf.drop(columns=['Country'], inplace=True)  # Clean up any extra columns

In [23]:
merged_gdf

,ADMIN,ISO_A3,ISO_A2,geometry,ratings
0,Aruba,ABW,AW,"MULTIPOLYGON (((-69.99694 12.57758, -69.93639 ...",NaN
1,Afghanistan,AFG,AF,"MULTIPOLYGON (((71.04980 38.40866, 71.05714 38...",NaN
2,Angola,AGO,AO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -...","[{'Year': 1960, 'Data': {'Arabica Production':..."
3,Anguilla,AIA,AI,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ...",NaN
4,Albania,ALB,AL,"MULTIPOLYGON (((19.74777 42.57890, 19.74601 42...","[{'Year': 1960, 'Data': {'Arabica Production':..."
...,...,...,...,...,...
250,Samoa,WSM,WS,"MULTIPOLYGON (((-171.57002 -13.93816, -171.564...",NaN
251,Yemen,YEM,YE,"MULTIPOLYGON (((53.30824 12.11839, 53.31027 12...","[{'Year': 1960, 'Data': {'Arabica Production':..."
252,South Africa,ZAF,ZA,"MULTIPOLYGON (((37.86378 -46.94085, 37.83644 -...","[{'Year': 1960, 'Data': {'Arabica Production':..."
253,Zambia,ZMB,ZM,"MULTIPOLYGON (((31.11984 -8.61663, 31.14102 -8...","[{'Year': 1960, 'Data': {'Arabica Production':..."


In [27]:
merged_gdf['ratings'] = merged_gdf['ratings'].apply(json.dumps)

In [25]:
merged_gdf

,ADMIN,ISO_A3,ISO_A2,geometry,ratings
0,Aruba,ABW,AW,"MULTIPOLYGON (((-69.99694 12.57758, -69.93639 ...",NaN
1,Afghanistan,AFG,AF,"MULTIPOLYGON (((71.04980 38.40866, 71.05714 38...",NaN
2,Angola,AGO,AO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -...","[{'Year': 1960, 'Data': {'Arabica Production':..."
3,Anguilla,AIA,AI,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ...",NaN
4,Albania,ALB,AL,"MULTIPOLYGON (((19.74777 42.57890, 19.74601 42...","[{'Year': 1960, 'Data': {'Arabica Production':..."
...,...,...,...,...,...
250,Samoa,WSM,WS,"MULTIPOLYGON (((-171.57002 -13.93816, -171.564...",NaN
251,Yemen,YEM,YE,"MULTIPOLYGON (((53.30824 12.11839, 53.31027 12...","[{'Year': 1960, 'Data': {'Arabica Production':..."
252,South Africa,ZAF,ZA,"MULTIPOLYGON (((37.86378 -46.94085, 37.83644 -...","[{'Year': 1960, 'Data': {'Arabica Production':..."
253,Zambia,ZMB,ZM,"MULTIPOLYGON (((31.11984 -8.61663, 31.14102 -8...","[{'Year': 1960, 'Data': {'Arabica Production':..."


In [28]:
# Save to GeoJSON
merged_gdf.to_file('enhanced_countries_with_detailed_yearly_data.geojson', driver='GeoJSON')
merged_gdf

,ADMIN,ISO_A3,ISO_A2,geometry,ratings
0,Aruba,ABW,AW,"MULTIPOLYGON (((-69.99694 12.57758, -69.93639 ...",NaN
1,Afghanistan,AFG,AF,"MULTIPOLYGON (((71.04980 38.40866, 71.05714 38...",NaN
2,Angola,AGO,AO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -...","[{""Year"": 1960, ""Data"": {""Arabica Production"":..."
3,Anguilla,AIA,AI,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ...",NaN
4,Albania,ALB,AL,"MULTIPOLYGON (((19.74777 42.57890, 19.74601 42...","[{""Year"": 1960, ""Data"": {""Arabica Production"":..."
...,...,...,...,...,...
250,Samoa,WSM,WS,"MULTIPOLYGON (((-171.57002 -13.93816, -171.564...",NaN
251,Yemen,YEM,YE,"MULTIPOLYGON (((53.30824 12.11839, 53.31027 12...","[{""Year"": 1960, ""Data"": {""Arabica Production"":..."
252,South Africa,ZAF,ZA,"MULTIPOLYGON (((37.86378 -46.94085, 37.83644 -...","[{""Year"": 1960, ""Data"": {""Arabica Production"":..."
253,Zambia,ZMB,ZM,"MULTIPOLYGON (((31.11984 -8.61663, 31.14102 -8...","[{""Year"": 1960, ""Data"": {""Arabica Production"":..."
